In [1]:
from iexfinance import Stock
from iexfinance import get_available_symbols

In [223]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("/Users/linhefeng/config_spacy.yml"))

# Load the training data
training_data = load_data('/Users/linhefeng/demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhefeng/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/linhe

In [118]:
import re
# 用于匹配月和日的正则表达式
pattern_day = re.compile('[0-9]{1,2}[a-z]{2}')
pattern_num = re.compile('[0-9]{1,2}')
pattern_year = re.compile('[0-9]{4}')

In [119]:
from iexfinance import get_available_symbols

In [198]:
# 常量定义
company_dict = {'apple': 'AAPL', 'tesla': 'TSLA', 'microsoft': 'MSFT', 'alcoa': 'AA'}

# 月份转换字典
month_list = {}
month_list['January'] = '1'
month_list['February'] = '2'
month_list['March'] = '3'
month_list['April'] = '4'
month_list['May'] = '5'
month_list['June'] = '6'
month_list['July'] = '7'
month_list['August'] = '8'
month_list['September'] = '9'
month_list['October'] = '10'
month_list['November'] = '11'
month_list['December'] = '12'

INIT = 0
CLOSE = 1
TIME = 2
VOLUME = 3
VALUE = 4
PRICE = 5

# 状态跳转规则
policy = {
    (INIT, 'close'): (CLOSE, 'Searching...'),
    (INIT, 'none'): (INIT, 'You can ask me for what you want.'),
    (CLOSE, 'time'): (TIME, 'Searching for details...'),
    (CLOSE, 'close'): (CLOSE, 'Searching...'),
    (CLOSE, 'deny'): (CLOSE, 'Already delete. Searching...'),
    (INIT, 'volume'): (VOLUME, 'Searching...'),
    (VOLUME, 'time'): (TIME, 'Searching for details...'),
    (VOLUME, 'deny'): (VOLUME, 'Already delete. Searching...'),
    (VOLUME, 'volume'): (VOLUME, 'Searching...'),
    (INIT, 'value'): (VALUE, 'Searching...'),
    (VALUE, 'time'): (TIME, 'Searching for details...'),
    (VALUE, 'deny'): (VALUE, 'Already delete. Searching...'),
    (VALUE, 'value'): (VALUE, 'Searching...'),
    (INIT, 'price'): (PRICE, 'Searching...'),
    (PRICE, 'deny'): (PRICE, 'Already delete. Searching...'),
    (PRICE, 'price'): (PRICE, 'Searching...')
}

answer = []

In [199]:
# 将message传递给bot
def send_message(message, policy, state, params, intent_dict, answer):
    print("USER : {}".format(message))
    answer = []
    new_state, response, params, answer = respond(policy, state, message, params, intent_dict, answer)
    return new_state, params, answer

# 根据message进行回复
def respond(policy, state, message, params, intent_dict, answer):
    (new_state, response) = policy[(state, interpret(message))]
#     print("BOT : {}".format(response))
    answer.append(response)
    params, answer = analyse(message, params, intent_dict, answer)
    # 默认查询完特定时间的股票信息后，状态重置
    empty = {}
    if new_state == TIME:
        if params == empty:
            new_state = INIT
        else:
            new_state = state
    return new_state, response, params, answer

# 识别意图并将状态返回
def interpret(message):
    intent = interpreter.parse(message)['intent']['name']
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue
    # 根据意图返回对应状态
    if intent == 'close':
        return 'close'
    elif intent == 'time':
        return 'time' 
    elif intent == 'volume':
        return 'volume'
    elif intent == 'value':
        return 'value'
    elif intent == 'deny':
        return 'deny'
    elif intent == 'price':
        return 'price'
    else:
        return 'none'

In [227]:
from iexfinance import get_historical_data
from datetime import datetime

# 抽取实体，并对于message进行解析、回复
def analyse(message, params, intent_dict, answer):
    day = month = 0
    year = '2018'
    entities = interpreter.parse(message)['entities']
    intent = interpreter.parse(message)['intent']['name']
    words = [str(i) for i in message.split()]
    for i in range(len(words)):
        if words[i] == 'no' or words[i] == 'not':
            intent = 'deny'
            break
        else:
            continue
    # 查询意图是查询实时股价
    if intent == 'price':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
#             print("BOT : The price of {0} is {1}".format(params[company], stock_search.get_price()))
            answer.append("The price of {0} is {1}".format(params[company], stock_search.get_price()))
        intent_dict.append(intent)
    # 查询意图是查询股票收盘价
    if intent == 'close':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
#             print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
            answer.append("The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        intent_dict.append(intent)
    # 查询意图是查询股票交易额
    elif intent == 'volume':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
#             print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
            answer.append("The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
        intent_dict.append(intent)
    # 查询意图是查询股票市值
    elif intent == 'value':
        for ent in entities:
            params[str(ent['entity']) + '_' + str(ent['value'])] = str(ent['value'])
        for company in params:
            stock_search = Stock(company_dict[str(params[company])])
#             print("BOT : The value of {0} is {1}".format(params[company], 
#                                                           stock_search.get_financials()[0]['totalAssets']))
            answer.append("The value of {0} is {1}".format(params[company], 
                                                          stock_search.get_financials()[0]['totalAssets']))
        intent_dict.append(intent)
    # 查询意图是对于之前的一些查询条件进行否定
    elif intent == 'deny':
        flag = False
        for i in range(len(words)):
            words[i] = words[i].lower()
            if words[i] == 'no' or words[i] == 'not':
                flag = True
            key = 'company_' + str(words[i])
            if key in params and flag:
                params.pop(key)
        # 根据上一次状态进行相应回复
        if intent_dict[len(intent_dict)-1] == 'close':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
#                 print("BOT : The close price of {0} is {1}".format(params[company], stock_search.get_close()))
                answer.append("The close price of {0} is {1}".format(params[company], stock_search.get_close()))
        elif intent_dict[len(intent_dict)-1] == 'value':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
#                 print("BOT : The value of {0} is {1}".format(params[company], 
#                                                           stock_search.get_financials()[0]['totalAssets']))
                answer.append("The value of {0} is {1}".format(params[company], 
                                                          stock_search.get_financials()[0]['totalAssets']))
        elif intent_dict[len(intent_dict)-1] == 'volume':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
#                 print("BOT : The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
                answer.append("The volume of {0} is {1}".format(params[company], stock_search.get_volume()))
        elif intent_dict[len(intent_dict)-1] == 'price':
            for company in params:
                stock_search = Stock(company_dict[str(params[company])])
#                 print("BOT : The price of {0} is {1}".format(params[company], stock_search.get_price()))
                answer.append("The price of {0} is {1}".format(params[company], stock_search.get_price()))
    # 根据时间进行给定时间的股票信息查询
    elif intent == 'time':
        # 对于句子中的月和日进行解析
        tmp = [str(i) for i in message.split()]
        for i in range(len(tmp)):
            if tmp[i] in month_list:
                month = int(month_list[tmp[i]])
            if re.search(pattern_day, tmp[i]) is not None:
                day = int(pattern_num.findall(pattern_day.findall(tmp[i])[0])[0])
            if re.search(pattern_year, tmp[i]) is not None:
                year = int(pattern_year.findall(tmp[i])[0])
            else:
                year = 2018
                if year >= 2019:
#                     print('BOT : Sorry. Wrong about input date.')
                    answer.append('Sorry. Wrong about input date.')
                    return params, answer
        start = datetime(year, month, day)
        end = datetime(year, month, day)
        if int(month) < 10:
            month = '0' + str(month)
        if int(day) < 10:
            day = '0' + str(day)
        params['time'] = str(year) + '-' + str(month) + '-' + str(day)
        if intent_dict[len(intent_dict)-1] == 'value':
            for company in params:
                if company == 'time':
                    continue
                value_search = Stock(company_dict[str(params[company])])
                data = value_search.get_financials()
                for k in range(len(data)):
                    if data[k]['reportDate'] <= params['time']:
#                         print('BOT : The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
#                                                                data[k]['totalAssets'], 
#                                                                            intent_dict[len(intent_dict)-1]))
                        answer.append('The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
                                                               data[k]['totalAssets'], 
                                                                           intent_dict[len(intent_dict)-1]))
                        break
                    else:
                        continue
        else:
            for company in params:
                if company == 'time':
                    continue
                info = get_historical_data(company_dict[str(params[company])], 
                                         start, end)
                info_details = info[company_dict[params[company]]]
                if str(params['time']) not in info_details:
#                         print('BOT : Sorry. Wrong about input Date.')
                        answer.append('Sorry. Wrong about input Date.')
                        params['time'] = ''
                        print(answer)
                        return params, answer
                result = info_details[str(params['time'])][intent_dict[len(intent_dict)-1]]
#                 print('BOT : The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
#                                                                result, intent_dict[len(intent_dict)-1]))
                answer.append('The {3} of {0} at {1} is {2}.'.format(params[company], params['time'], 
                                                               result, intent_dict[len(intent_dict)-1]))
        
            params = {}
    print(answer)
    return params, answer


In [228]:
state = INIT
params = {}
intent_dict = []
answer = []
state, params, answer = send_message('hello', policy, state, params, intent_dict, answer)
state, params, answer = send_message('I would like to know the close price of Tesla and Apple.', 
                             policy, state, params, intent_dict, answer)
state, params, answer = send_message('not tesla', policy, state, params, intent_dict, answer)
state, params, answer = send_message("How about March 8th", policy, state, params, intent_dict, answer)
state, params, answer = send_message("I want to know the Apple's close price.", policy, state, params, intent_dict, answer)
state, params, answer = send_message("How about February 20th in 2017", policy, state, params, intent_dict, answer)
state, params, answer = send_message("How about February 21th in 2017", policy, state, params, intent_dict, answer)
state, params, answer = send_message("I want to know the price of Apple and Tesla", 
                             policy, state, params, intent_dict, answer)
state, params, answer = send_message("not Apple", policy, state, params, intent_dict, answer)
# state, params, answer = send_message("I want to know the close price of Apple.", policy, state, params, intent_dict, answer)
# state, params, answer = send_message("Tell me the close price of Tesla.", policy, state, params, intent_dict, answer)
# aapl.get_financials()

USER : hello
['You can ask me for what you want.']
USER : I would like to know the close price of Tesla and Apple.
['Searching...', 'The close price of tesla is 354.31', 'The close price of apple is 193.53']
USER : not tesla
['Already delete. Searching...', 'The close price of apple is 193.53']
USER : How about March 8th
['Searching for details...', 'The close of apple at 2018-03-08 is 175.0336.']
USER : I want to know the Apple's close price.
['Searching...', 'The close price of apple is 193.53']
USER : How about February 20th in 2017
['Searching for details...', 'Sorry. Wrong about input Date.']
USER : How about February 21th in 2017
['Searching for details...', 'The close of apple at 2017-02-21 is 133.1211.']
USER : I want to know the price of Apple and Tesla
['Searching...', 'The price of apple is 193.53', 'The price of tesla is 354.31']
USER : not Apple
['Already delete. Searching...', 'The price of tesla is 354.31']



An error occurred in <function reply_my_friend at 0xd31412b70>.
Traceback (most recent call last):
  File "/Users/linhefeng/anaconda3/lib/python3.6/site-packages/wxpy/api/bot.py", line 326, in process
    ret = config.func(msg)
  File "<ipython-input-226-708017162d91>", line 5, in reply_my_friend
    result = test(msg)
  File "<ipython-input-225-078a649b49da>", line 8, in test
    state, params, answer = send_message(str(msg.text), policy, state, params, intent_dict, answer)
  File "<ipython-input-199-313ba6aa132e>", line 5, in send_message
    new_state, response, params, answer = respond(policy, state, message, params, intent_dict, answer)
  File "<ipython-input-199-313ba6aa132e>", line 10, in respond
    (new_state, response) = policy[(state, interpret(message))]
KeyError: (5, 'volume')


USER : 嗯嗯


In [230]:
from wxpy import *

bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Mccpiggy


In [231]:
#coding=utf8
my_friend = bot.friends().search('孙誉铭', sex=FEMALE)
print(my_friend)

[<Friend: 孙誉铭>]


In [237]:
state = INIT
params = {}
intent_dict = []
answer = []

def test(msg):
    global state, params, answer
    state, params, answer = send_message(str(msg.text), policy, state, params, intent_dict, answer)
    return answer

In [238]:
@bot.register(my_friend)
def reply_my_friend(msg):
#     global state, params, answer
#     state, params, answer = send_message(str(msg.text), policy, state, params, intent_dict, answer)
    result = test(msg)
    for i in range(len(result)):
        msg.reply(result[i])
bot.join()

USER : hello
['You can ask me for what you want.']
USER : I want to know the close price of Tesla and Apple.
['Searching...', 'The close price of tesla is 354.31', 'The close price of apple is 193.53']
USER : not Tesla
['Already delete. Searching...', 'The close price of apple is 193.53']
USER : How about March 8th?
['Searching for details...', 'The close of apple at 2018-03-08 is 175.0336.']
USER : I want to know the Apple's close price.
['Searching...', 'The close price of apple is 193.53']
USER : How about February 20th in 2017?
['Searching for details...', 'Sorry. Wrong about input Date.']
USER : How about February 21th in 2017?
['Searching for details...', 'The close of apple at 2017-02-21 is 133.1211.']
USER : I want to know the price of Apple and Tesla.
['Searching...', 'The price of apple is 193.53', 'The price of tesla is 354.31']
USER : not Apple
['Already delete. Searching...', 'The price of tesla is 354.31']


In [229]:
bot.logout()

LOG OUT!


<ItchatReturnValue: {'BaseResponse': {'ErrMsg': '请求成功', 'Ret': 0, 'RawMsg': 'logout successfully.'}}>